In [1]:
def pagerank(G, alpha=0.85, personalization=None,
             max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
             dangling=None):

    if len(G) == 0:
        return {}
 
    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G
 
    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()
 
    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())
 
    if personalization is None:
 
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise TypeError('Personalization dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())
 
    if dangling is None:
 
        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise TypeError('Dangling node dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
 
    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
 
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]
 
        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise Exception('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter)

In [2]:
import networkx as nx
G=nx.barabasi_albert_graph(60,41)
pr=pagerank(G,0.85)
pr

{0: 0.010825943316893401,
 1: 0.010818899312186325,
 2: 0.010303079305546008,
 3: 0.010823537700642379,
 4: 0.009255650451847114,
 5: 0.010820856040845537,
 6: 0.010300485182388555,
 7: 0.009254872054161924,
 8: 0.010821871298796568,
 9: 0.009250486625678522,
 10: 0.011343961916808485,
 11: 0.009776538197601062,
 12: 0.00873470809623846,
 13: 0.009780492187344624,
 14: 0.01081712376719075,
 15: 0.009260408099753686,
 16: 0.010299785767225978,
 17: 0.01186356554136035,
 18: 0.010301227827728533,
 19: 0.010302074794325664,
 20: 0.01134173497316942,
 21: 0.011861502852029141,
 22: 0.010304629328543436,
 23: 0.0102974126580175,
 24: 0.008738741488213514,
 25: 0.01081870496205224,
 26: 0.011343553893041102,
 27: 0.011341288268454005,
 28: 0.009261185032039824,
 29: 0.00977287138320625,
 30: 0.009777419795750624,
 31: 0.007188200179486854,
 32: 0.0087322653327877,
 33: 0.008219271490962791,
 34: 0.010823510527940883,
 35: 0.011341770077250133,
 36: 0.01082076765138345,
 37: 0.009260084724800

In [3]:
new_res = {k: v for k, v in sorted(pr.items(), key=lambda item: item[1], reverse = True)}

In [4]:
new_res

{41: 0.035989218424079025,
 42: 0.035311289635847044,
 43: 0.0347650032537335,
 44: 0.0341850922296011,
 45: 0.03354577737875282,
 46: 0.03296503271468112,
 47: 0.03241077668826072,
 48: 0.031699283808604456,
 49: 0.03118825113647204,
 50: 0.0306100449151548,
 51: 0.030081398593432295,
 52: 0.02934366479014085,
 53: 0.028771807731190483,
 54: 0.028252442625631317,
 55: 0.027632512015042753,
 56: 0.0270456568255078,
 57: 0.02651752944219974,
 58: 0.02583374793898468,
 59: 0.025184551554043123,
 17: 0.01186356554136035,
 21: 0.011861502852029141,
 40: 0.011344875387045823,
 39: 0.011343993921673318,
 10: 0.011343961916808485,
 26: 0.011343553893041102,
 35: 0.011341770077250133,
 20: 0.01134173497316942,
 27: 0.011341288268454005,
 0: 0.010825943316893401,
 3: 0.010823537700642379,
 34: 0.010823510527940883,
 8: 0.010821871298796568,
 5: 0.010820856040845537,
 36: 0.01082076765138345,
 1: 0.010818899312186325,
 25: 0.01081870496205224,
 14: 0.01081712376719075,
 22: 0.010304629328543436,